![logo](../LogoLine_horizon_C3S.png)

# Deriving the ENSO indices used in graphical products from CDS data

In this example we how to compute the El Niño–Southern Oscillation (ENSO) indices used in [C3S seasonal graphical products](https://climate.copernicus.eu/charts/packages/c3s_seasonal/), from SST data retrieved from the CDS. 

This is shown for one seasonal forecast system, but can be repeated to provide input for a further example on computing the correlation heatmaps displayed for the SST indices on the [verification page](https://confluence.ecmwf.int/display/CKB/C3S+seasonal+forecasts+verification+plots) associated with the graphical products.

Finally, some information on ENSO impacts in Europe can be found on [a page in the C3S documentation](https://confluence.ecmwf.int/display/COPSRV/ENSO+impacts+on+Europe).